In [ ]:
import pandas as pd
from modules import *
from schemaVR import custom_schema
%matplotlib widget

### 1. Load dataset

In [ ]:
datapicker = create_datapicker(show_summary=False, ubx=False, create_geodata=False, schema=custom_schema)
display(datapicker)

In [ ]:
datapicker.dataset.streams.Unity.Georeference.data


In [ ]:
space_time = datapicker.dataset.streams.Unity.Transform.data.copy()
time = datapicker.dataset.streams.Unity.Transform.data.Timestamp
space_time.index = time.index
space_time
#spacetime = pd.DataFrame({'Latitude': space["Transform.Position.X"],
#                          'Longitude': space["Transform.Position.Z"],
#                          'Elevation': space["Transform.Position.Y"]})
#spacetime.index = time.index
#georeference = Georeference(spacetime=spacetime)
# dataset.streams.Unity.Transform.Timestamps.data

In [ ]:

sampling_dt=datetime.timedelta(seconds = 2)
input_data = datapicker.dataset.streams.BioData.ECG.data

# resampled
ecg_correlation = pd.merge_asof(input_data, space_time, left_index=True, right_index=True)
ecg_resample = ecg_correlation.resample(sampling_dt, origin='start').mean()

# input_data = datapicker.dataset.streams.EEG.Sync.data
# eeg_correlation = pd.merge_asof(input_data, space_time, left_index=True, right_index=True)

# Some possible plots
# eeg_correlation.plot(x='Latitude',y='Longitude')
# resample
#eeg_correlation.plot(x='Latitude', y='Longitude')
# fig = plt.figure()
# ecg_correlation.Value0.plot()
# ecg_resample.Value0.plot()

ecg_correlation

### 2. Explore Dataset Maps

In [ ]:
exploremap(datapicker.geodata, column='tk_airquality_iaqindex_value')

### 3. Explore Dataset Streams

In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
    'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
})

### 4. Export Dataset to OGC API

In [ ]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> Outdoor Walk: <CityRegion> Subject <ID>',
    description='Outdoor walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'Outdoor', 'Walk', 'Microclimate', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [ ]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())